**Application of Once Class SVM to detect anomaly** 

In [464]:
import pandas as pd
import numpy as np
# import kagglehub
from sklearn import svm
# processed_path = kagglehub.dataset_download('/kaggle/input/processed/output_file.xlsx')
cc=pd.read_excel('/kaggle/input/processed/output_file.xlsx')

In [465]:
# Data check. 

cc.head()

,bht,age,edu,wrk,mrt,gaus,grav,para,bmi,mul,...,amnfle,hyste,seps,complot,fnl,ptdest,length_of_labor,nrms_1,nrms_2,nrms_
0,0.054074,1.631284,-0.455792,0.442040,-0.076584,0,1.649222,2.109449,1.431228,-0.054074,...,0.0,-0.054153,-0.121806,-0.133631,-0.127189,-0.134256,10.717942,0.154533,-0.144338,-0.054074
1,0.054074,0.238984,-0.455792,0.442040,-0.076584,0,-0.906576,-0.880390,-0.900892,-0.054074,...,0.0,-0.054153,-0.121806,-0.133631,-0.127189,0.154522,7.974096,0.154533,-0.144338,-0.054074
2,0.054074,1.631284,2.193983,-2.172095,-0.076584,0,-0.054643,0.116223,0.265168,-0.054074,...,0.0,-0.054153,-0.121806,-0.133631,-0.127189,8.529081,7.974096,0.154533,-0.144338,-0.054074
3,0.054074,1.109171,-0.455792,0.442040,-0.076584,0,1.649222,1.112836,0.265168,-0.054074,...,0.0,18.466185,-0.121806,-0.133631,-0.127189,0.154522,2.735846,0.154533,-0.144338,-0.054074
4,0.054074,0.935134,-0.455792,0.442040,-0.076584,0,-0.906576,-0.880390,-0.900892,-0.054074,...,0.0,-0.054153,-0.121806,-0.133631,-0.127189,1.020855,2.985286,0.154533,-0.144338,-0.054074


In [466]:
features=['gdm', 'precs', 'nrprecs', 'bmi', 'mac','prebrth']
cc=cc[features]

In [467]:
#I observed an conflict in the name 'class'. Therefore, I have changed the name from class to category

cc= cc.rename(columns={'prebrth': 'Category'})

In [468]:
#Class 2 & 3 were combined and standardised class to (0,1)

# Replace values in the 'Category' column
cc['Category'] = cc['Category'].replace({3: 2})

# Re-label classes to start from 0
cc['Category'] = cc['Category'] - 1  # Convert {1, 2} -> {0, 1}
print(cc['Category'].value_counts())

Category
0    327
1     16
Name: count, dtype: int64


In [469]:
# For convinience, divide the dataframe cc based on two labels. 

nor_obs = cc.loc[cc.Category==0]    #Data frame with normal observation
ano_obs = cc.loc[cc.Category==1]    #Data frame with anomalous observation

In [470]:
print('nor_obs',nor_obs.shape)
print('ano_obs',ano_obs.shape)

nor_obs (327, 6)
ano_obs (16, 6)


The given dataframe 'cc' is divided into three sets 

Training set: train_features

Test observations/features: X_test

Test labels: Y_test

Once class SVM is trained with the observations of only one class. In this case, the algorithm is trained with first 200,000 observation of normal transactions. The remaining observations are merged with the anomalous observation to create a test set. 



In [471]:
# The given dataframe 'cc' is divided into three sets 
# Training set: train_features
# Test observations/features: X_test
# Test labels: Y_test

In [472]:
# Once class SVM is trained with the observations of only one class. In this case, the algorithm is trained with 
# first 200,000 observation of normal transactions. The remaining observation is merged with the anomalous observation 
# to create a test set. 

train_feature = nor_obs.loc[0:335, :]
train_feature = train_feature.drop('Category', axis=1)
Y_1 = nor_obs.loc[335:, 'Category']
Y_2 = ano_obs['Category']

In [473]:
# Creatng test observations/features

X_test_1 = nor_obs.loc[335:, :].drop('Category',axis=1)
X_test_2 = ano_obs.drop('Category',axis=1)
X_test = pd.concat([X_test_1,X_test_2])

In [474]:
print(X_test)

          gdm     precs   nrprecs       bmi       mac
335 -0.398477 -0.478947 -0.439549  0.265168 -0.242161
336 -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
337 -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
338 -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
339 -0.398477  2.087912  1.544204  0.265168 -0.242161
340 -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
341 -0.398477 -0.478947 -0.439549  0.265168 -0.242161
342 -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
25  -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
42  -0.398477 -0.478947 -0.439549 -0.900892 -0.242161
52  -0.398477 -0.478947 -0.439549  1.431228 -0.242161
55  -0.398477 -0.478947 -0.439549 -2.066951 -0.242161
64   2.509556 -0.478947 -0.439549  0.265168 -0.242161
66  -0.398477  2.087912  1.544204  1.431228 -0.242161
118 -0.398477 -0.478947 -0.439549  1.431228 -0.242161
145 -0.398477  2.087912  1.544204  0.265168 -0.242161
148 -0.398477  2.087912  1.544204  1.431228 -0.242161
174 -0.398477 -0.478947 -0.4

In [475]:
# Setting the hyperparameters for Once Class SVM

oneclass = svm.OneClassSVM(kernel='rbf', gamma=0.35, nu=0.35)

# I have used various combination of hyperparameters like linear, rbf, poly, gamma- 0.001, 0.0001, nu- 0.25, 0.5, 0.75, 0.95
# This combination gave me the most satisfactory results.# The remain data set is (after 200,000 observations) are appended with anomalous observations

Y_1 = nor_obs.loc[335:, 'Category']
Y_2 = ano_obs['Category']

Y_test= pd.concat([Y_1,Y_2])

#Y_test is used to evaluste the model

In [476]:
from sklearn import svm
# Training the algorithm with the features. 
# This stage is very time consuming processes. In my laptop it took more than an hour to train for 200,000 observations. 
# For rbf, the time taken is even more.

oneclass.fit(train_feature)

OneClassSVM(gamma=0.35, nu=0.35)

In [477]:
# Test the algorithm on the test set

fraud_pred = oneclass.predict(X_test)

In [478]:
# Check the number of outliers predicted by the algorithm

unique, counts = np.unique(fraud_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

[[-1 11]
 [ 1 13]]


In [479]:
#Convert Y-test and fraud_pred to dataframe for ease of operation

Y_test= Y_test.to_frame()
Y_test=Y_test.reset_index()
fraud_pred = pd.DataFrame(fraud_pred)
fraud_pred= fraud_pred.rename(columns={0: 'prediction'})

In [480]:
##Performance check of the model

TP = FN = FP = TN = 0
for j in range(len(Y_test)):
    if Y_test['Category'][j]== 1 and fraud_pred['prediction'][j] == -1:
        TP = TP+1
    elif Y_test['Category'][j]== 1 and fraud_pred['prediction'][j] == 1:
        FN = FN+1
    elif Y_test['Category'][j]== 0 and fraud_pred['prediction'][j] == -1:
        FP = FP+1
    else:
        TN = TN +1
print (TP,  FN,  FP,  TN)


10 6 1 7


poly 5: 16 0 8 0    param:kernel='poly', degree=5, gamma=0.1, nu=0.01
linear: 8 8 1 7      param:kernel='linear', gamma=0.1, nu=0.01
rbf: 0 16 0 8    param:kernel='rbf', gamma=0.1, nu=0.01
rbf: 10 6 3 5    param: kernel='rbf', gamma=0,35, nu=0.01
rbf: 4 12 1 7    param: kernel=rbf. gamma=0.55, nu=0,01
rbf: 10 6 1 7    param: kernel='rbf', gamma=0,35, nu=0.35


In [481]:
# Performance Matrix

accuracy = (TP+TN)/(TP+FN+FP+TN)
print (accuracy)
sensitivity = TP/(TP+FN)
print (sensitivity)
specificity = TN/(TN+FP)
print (specificity)

0.7083333333333334
0.625
0.875


Following results were obtained 

accuracy= 99.9%

sensitivity = 100%

specificity = 75%

Once class SVM has shown a very promising performance for this dataset with near 90% detection of anomaly and very few false alarm. This can be a starting point for fine tuning the algorthm to improve the specificity, keeping other things constant. Tuning the hyperparameters are very time consuming process and the Kaggle kernal stops after some time. Therefore, O couldnt run the code. I have just shown my codes in the cell. I am sure this code will run because i have ran it in my Jupyter note book. I have also isolation forest in my previous kernal. Once class SVM seems to outperform isolation forest in this case. 